In [ ]:
!pip install langchain
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cach

NOTE: You might need to restart the session after the pip install block

In [ ]:
import warnings
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers.models.mistral.modeling_mistral import MistralForCausalLM
from transformers.models.llama.tokenization_llama_fast import LlamaTokenizerFast
from transformers import LlamaForCausalLM
from transformers import PreTrainedTokenizerFast

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# for blocked repo, you need huggingface.login()
import huggingface_hub
huggingface_hub.login()

In [ ]:
model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization_config=quantization_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from typing import Optional, List, Mapping, Any

In [ ]:
class CustomLLMLlama(LLM):
    model: LlamaForCausalLM
    tokenizer: PreTrainedTokenizerFast

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None) -> str:

        messages = [
         {"role": "user", "content": prompt},
        ]

        terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        encodeds = self.tokenizer.apply_chat_template(messages, return_tensors="pt")
        model_inputs = encodeds.to(self.model.device)

        generated_ids = self.model.generate(model_inputs, max_new_tokens=512, eos_token_id=terminators, do_sample=True, pad_token_id=tokenizer.eos_token_id, top_k=4, temperature=0.7)
        response = generated_ids[0][model_inputs.shape[-1]:]
        output = tokenizer.decode(response, skip_special_tokens=True)

        if stop is not None:
          for word in stop:
            output = output.split(word)[0].strip()

        return output

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": self.model}

In [ ]:
llm = CustomLLMLlama(model=model, tokenizer=tokenizer)

### Tools

In [ ]:
!pip install wikipedia
!pip install --upgrade --quiet  wikibase-rest-api-client mediawikiapi
!pip install --upgrade stackapi

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=6e5834c451ed4454bd61bf74b2733984e85bf89bae8cf564328032ebee7ade4b
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [ ]:
import numexpr as ne
from langchain.tools import WikipediaQueryRun, BaseTool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain_community.utilities import StackExchangeAPIWrapper
from langchain.agents import Tool

In [ ]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=2500))
print(wikipedia.run("Deep Learning"))

Page: Deep learning
Summary: Deep learning is the subset of machine learning methods based on neural networks with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.
Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
Early forms of neural networks were inspired by information processing and distributed communication nodes in biological systems, in particular the human brain. However, current neural networks do not intend to model t

In [ ]:
wikidata = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())
print(wikidata.run("Alan Turing"))

Result Q7251:
Label: Alan Turing
Description: English computer scientist (1912–1954)
Aliases: Alan M. Turing, Alan Mathieson Turing, Turing, Alan Mathison Turing
instance of: human
country of citizenship: United Kingdom
occupation: computer scientist, mathematician, university teacher, cryptographer, logician, statistician, marathon runner, artificial intelligence researcher
sex or gender: male
date of birth: 1912-06-23
date of death: 1954-06-07
sport: athletics
place of birth: Maida Vale, Warrington Lodge
educated at: King's College, Princeton University, Sherborne School, Hazlehurst Community Primary School
employer: Victoria University of Manchester, Government Communications Headquarters, University of Cambridge, National Physical Laboratory
place of death: Wilmslow
field of work: cryptanalysis, computer science, mathematics, logic, cryptography
cause of death: cyanide poisoning
notable work: On Computable Numbers, with an Application to the Entscheidungsproblem, Computing Machiner

In [ ]:
stackexchange = StackExchangeAPIWrapper()

stackexchange.run("zsh: command not found: python")

'Question: how to know what commands running in background on windows?\nQuite while ago, I saw someone using <span class="highlight">python</span> script to show background commands running on his computer while malicious doc file are opening. where can I find something like that? &hellip; \n\nQuestion: ROS_getting from publisher data\ni have problem when i get the data from pwm_value.\n<span class="highlight">command</span> &quot;rostopic echo pwm_value&quot; they show just 50(i means that the msg.data =0 probably becuase of mapping function)\nmsg.data is from ROS(linear.x , &hellip; angular.z) i got that data from ros <span class="highlight">python</span> file (it doesn&#39;t matter when i check getting the data )\nso it seems like because of my code logic.... &hellip; \n\nQuestion: Missing endpoints in FastAPI Swagger UI\n<span class="highlight">command</span>: sh -c &quot;alembic upgrade head &amp;&amp; python3 main.py&quot;\n    ports:\n      - &quot;8000:8000&quot;\n    restart: 

In [ ]:
wikipedia_tool = Tool(
    name="Wikipedia",
    description="This tool queries Wikipedia, an online encyclopedia, created and edited by volunteers around the world. Never search for more than one concept at a single step. If you need to compare two concepts, search for each one individually. Syntax: string with a simple concept",
    func=wikipedia.run
)

In [ ]:
wikidata_tool = Tool(
    name="wikidata",
    description="This tool queries Wikidata, a free and open knowledge base. Designed for single-concept searches, input should be a simple, clear string representing one concept only. For comparisons or multiple concepts, conduct separate searches for each concept.",
    func=wikidata.run
)

In [ ]:
stackexchange_tool = Tool(
    name="stackexchange",
    description="Tool for querying computer programming questions. Search for expert answers and discussions related to various programming challenges and topics.",
    func=stackexchange.run
)

In [ ]:
# custom tool
class Calculator(BaseTool):
    name = "calculator"
    description = "This tool is only for mathematical questions and nothing else. Only input numexpr syntax."
    #"Never use this tool except for mathematical operations. It requires numexpr syntax. Use it always you need to solve any math operation. Be sure syntax is correct."

    def _run(self, expression: str):
      try:
        return ne.evaluate(expression).item()
      except Exception:
        return "This is not a numexpr valid syntax. Try a different syntax."

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

In [ ]:
calculator_tool = Calculator()
calculator_tool.run("2+3")

5

In [ ]:
tools = [wikidata_tool, calculator_tool, wikipedia_tool]

### Customizing the Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
system = """You are a helpful assistant trying to answer questions based on your knowledge.
Each task involves multiple steps, represented by a markdown code snippet of a json blob.
JSON Structure Requirements:
thought -> your thoughts for the next step
action -> name of a tool to use
action_input -> parameters required for the tool

Attempt to answer the questions without tools, using your knowledge first. In case you can't answer you have access to the following tools: {tool_names}.

Use tools only when the problem matches the tool descriptions:

{tools}.

Guidelines for tool usage:
For questions that are illogical or factually inconsistent, please use the tool 'Final Answer' to provide a reasoned explanation rather than attempting an incorrect or misleading answer.
Try to answer the questions as best as you can first. Tools are not mandatory for every task.
If tools are not valid, discontinue their use and respond using the 'Final Answer' tool based on your own knowledge.
If you don't know the answer nor can find a relevant tool to help you, clearly state that you won't be able to help with that problem.

Response Strategies:
If sufficiently informed, use the tool 'Final Answer' with the answer in as its parameter.
If there is not enough information, and all relevant tools have been exhausted without success, acknowledge the limitations of the available data or tools.
"""

In [ ]:
human="""
Add the word "STOP" after each JSON markdown snippet. Example:

```json
{{"thought": "<your thoughts>",
 "action": "<tool name or Final Answer to give a final answer>",
 "action_input": "<tool parameters or the final output"}}
```
STOP

This is my query="{input}". Write only the next step needed to solve it.
Base your answer on previous tool executions only when they have proven useful and relevant to the current query.
Remember to add STOP after each snippet.

These were the previous steps given to solve this query and the information you already gathered:
"""

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human),
        MessagesPlaceholder("agent_scratchpad")
    ]
)

### Building the Agent

In [ ]:
from langchain.agents import create_json_chat_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory

In [ ]:
agent = create_json_chat_agent(
    tools = tools,
    llm = llm,
    prompt = prompt,
    stop_sequence = ["STOP"],
    template_tool_response = "{observation}"
)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, memory=memory)
# def _handle_error(error) -> str:
#     if isinstance(error, IndexError):
#         return "Error: List index out of range. Ensure your data contains expected delimiters."
#     else:
#         return f"{type(error).__name__}: {str(error)[:50]}..."


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True) # handle_parsing_errors=_handle_error)

### Tests

In [ ]:
agent_executor.invoke({"input": "What time is it?"})



> Entering new AgentExecutor chain...
assistant

Here is the next step:

```json
{"thought": "We need to figure out how to get the current time.",
 "action": "Final Answer",
 "action_input": "We can't get the current time without a tool, so we need to use a tool to get the current time."}

> Finished chain.


{'input': 'What time is it?',
 'output': "We can't get the current time without a tool, so we need to use a tool to get the current time."}

In [ ]:
llm(prompt="what time is it?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"assistant\n\nI'm just an AI, I don't have access to real-time information about the current time. I can provide you with a virtual clock that updates in real-time, but it won't reflect the actual time where you are. Would you like me to provide a virtual clock?"

In [ ]:
agent_executor.invoke({"input": "What is the capital of Seoul?"})



> Entering new AgentExecutor chain...
assistant

```json
{"thought": "I need to find the capital of Seoul, but I'm not sure where to start.",
 "action": "Wikipedia",
 "action_input": "Seoul"}Page: Seoul
Summary: Seoul, officially Seoul Special City, and formerly known as Hanseong and Keijō, is the capital of the Republic of Korea (ROK), commonly known as South Korea, and the country's most extensive urban center. The broader Seoul Capital Area, encompassing Gyeonggi province and Incheon metropolitan city, emerged as the world's fourth largest metropolitan economy in 2014, trailing only Tokyo, New York City, and Los Angeles, hosting more than half of South Korea's population. Although Seoul's population peaked at slightly over 10 million, it has gradually decreased since 2014, standing at approximately 9.97 million residents as of 2020. Seoul is the seat of the South Korean government.
Seoul's history traces back to 18 BC when it was founded by the people of Baekje, one of the Three K

{'input': 'What is the capital of Seoul?',
 'output': 'The capital of South Korea is Seoul'}

In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...
assistant

Here's the next step:

```json
{"thought": "To solve this problem, I need to calculate the sum of twelve and three, then raise 3 to the fifth power and multiply the two results together.",
 "action": "calculator",
 "action_input": "12 + 3"}15assistant

```json
{"thought": "Now that I have the sum of twelve and three, which is 15, I need to raise 3 to the fifth power.",
 "action": "calculator",
 "action_input": "3 ** 5"}243assistant

```json
{"thought": "Now that I have 3 to the fifth power, which is 243, I need to multiply the sum of twelve and three by this result.",
 "action": "calculator",
 "action_input": "243 * 15"}3645assistant

```json
{"thought": "Now that I have the result of multiplying the sum of twelve and three by 3 to the fifth power, which is 3645, I need to square the whole result.",
 "action": "calculator",
 "action_input": "3645 ** 2"}13286025assistant

```json
{"thought": "I need to calculate the square of the whole 

{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': '13286025.0'}

In [ ]:
agent_executor.invoke({"input": "how can i fix the error zsh: command not found: python"})



> Entering new AgentExecutor chain...
assistant

Here's the next step:

```json
{"thought": "The error message suggests that the system is unable to find the Python executable. Let's check the system's PATH environment variable to see if Python is included.",
 "action": "calculator",
 "action_input": "PATH"}
```This is not a numexpr valid syntax. Try a different syntax.assistant

```json
{"thought": "The error message is suggesting that the system is unable to find the Python executable, so let's check the system's PATH environment variable to see if Python is included.",
 "action": "wikipedia",
 "action_input": "PATH environment variable"}
```Page: PATH (variable)
Summary: PATH is an environment variable on Unix-like operating systems, DOS, OS/2, and Microsoft Windows, specifying a set of directories where executable programs are located. In general, each executing process or user session has its own PATH setting.assistant

```json
{"thought": "The PATH environment variable seems to

{'input': 'how can i fix the error zsh: command not found: python',
 'output': ''}

In [ ]:
agent_executor.invoke({"input": "How much is 23 plus 17?"})



> Entering new AgentExecutor chain...
assistant

```json
{"thought": "I need to perform arithmetic operations to get the answer",
 "action": "calculator",
 "action_input": "23+17"}40assistant

```json
{"thought": "I need to evaluate the arithmetic expression",
 "action": "calculator",
 "action_input": "23+17"}40assistant

```json
{"thought": "The result is a simple arithmetic calculation",
 "action": "calculator",
 "action_input": "23+17"}40assistant

```json
{"thought": "The calculation is correct, now I need to provide the answer",
 "action": "Final Answer",
 "action_input": "40"}

> Finished chain.


{'input': 'How much is 23 plus 17?', 'output': '40'}

In [ ]:
agent_executor.invoke({"input": "What is the capital of France?"})



> Entering new AgentExecutor chain...
assistant

Here's the next step:

```json
{"thought": "I'm not sure about the capital of France, but I know it's a country in Europe.",
 "action": "Wikipedia",
 "action_input": "France"}Page: France
Summary: France, officially the French Republic, is a country located primarily in Western Europe. It also includes overseas regions and territories in the Americas and the Atlantic, Pacific and Indian oceans, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the north east, Switzerland to the east, Italy and Monaco to the south east, Andorra and Spain to the south, and a maritime border with the United Kingdom to the north west. Its metropolitan area extends from the Rhine to the Atlantic Ocean and from the Mediterranean Sea to the English Channel and the North Sea. Its overseas territories include French Guiana in South America, Sa

{'input': 'What is the capital of France?', 'output': 'Paris'}

In [ ]:
agent_executor.invoke({"input": "Who was the inventor of the Radio?"})



> Entering new AgentExecutor chain...
assistant

Here is the next step to solve the query:

```json
{"thought": "I think the inventor of the Radio is Guglielmo Marconi, but I'm not sure yet.",
 "action": "Wikipedia",
 "action_input": "Guglielmo Marconi"}
```Page: Guglielmo Marconi
Summary: Guglielmo Giovanni Maria Marconi, 1st Marquis of Marconi  (Italian: [ɡuʎˈʎɛlmo marˈkoːni]; 25 April 1874 – 20 July 1937) was an Italian inventor and electrical engineer, known for his creation of a practical radio wave–based wireless telegraph system. This led to Marconi's being credited as the inventor of radio, and he shared the 1909 Nobel Prize in Physics with Karl Ferdinand Braun "in recognition of their contributions to the development of wireless telegraphy".
Marconi was also an entrepreneur, businessman, and founder of The Wireless Telegraph & Signal Company in the United Kingdom in 1897 (which became the Marconi Company). In 1929, Marconi was ennobled as a Marchese (marquis) by King Victor 

{'input': 'Who was the inventor of the Radio?',
 'output': "Marconi's invention was the foundation of radio"}

In [ ]:
agent_executor.invoke({"input": "What is the double of the population of Madrid?"})



> Entering new AgentExecutor chain...
assistant

```json
{"thought": "I need to find the population of Madrid. I can try to use Wikipedia to get this information.",
 "action": "wikipedia",
 "action_input": "Madrid"}Page: Madrid
Summary: Madrid ( mə-DRID, Spanish: [maˈðɾið] ) is the capital and most populous city of Spain. The city has almost 3.4 million inhabitants and a metropolitan area population of approximately 7 million. It is the second-largest city in the European Union (EU), and its monocentric metropolitan area is the second-largest in the EU. The municipality covers 604.3 km2 (233.3 sq mi) geographical area. Madrid lies on the River Manzanares in the central part of the Iberian Peninsula at about 650 meters above mean sea level. The capital city of both Spain and the surrounding autonomous community of Madrid (since 1983), it is also the political, economic, and cultural centre of the country. The climate of Madrid features hot summers and cool winters.
The Madrid urban ag

{'input': 'What is the double of the population of Madrid?',
 'output': 'The double of the population of Madrid is 6.8 million'}

The following example usually fails due to limits in the model's reasoning capacity

In [ ]:
agent_executor.invoke({"input": "Who is older, Tom Hanks or Kevin Costner?"})



> Entering new AgentExecutor chain...
assistant

```json
{"thought": "I need to find the birthdate of Tom Hanks and Kevin Costner to determine who is older.",
 "action": "Wikipedia",
 "action_input": "Tom Hanks"}Wikipedia is not a valid tool, try one of [wikidata, calculator, wikipedia].assistant

```json
{"thought": "I'll try searching for Tom Hanks' birthdate on Wikidata.",
 "action": "wikidata",
 "action_input": "Tom Hanks"}Result Q2263:
Label: Tom Hanks
Description: American actor and film producer (born 1956)
Aliases: Thomas Jeffrey Hanks
instance of: human
country of citizenship: United States of America, Greece
occupation: film actor, film director, film producer, writer, voice actor, screenwriter, composer, character actor, television actor, television director, television producer, actor, television presenter, executive producer, narrator, stage actor, comedian, short story writer, director
sex or gender: male
date of birth: 1956-07-09
place of birth: Concord
educated at: Ch

{'input': 'Who is older, Tom Hanks or Kevin Costner?',
 'output': "Tom Hanks' birthdate (1956-07-09) is older than Kevin Costner's birthdate (1955-01-18)"}

### QA dataset

In [ ]:
res = agent_executor.invoke({"input": "한국의 수도는 어디야?"})

TypeError: Chain.invoke() takes from 2 to 3 positional arguments but 11 were given

In [ ]:
res['output']

'The capital of South Korea is Seoul'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os

In [ ]:
def get_json_list(file_path):
    file_path = os.path.expanduser(file_path)
    file_extension = file_path.split('.')[-1]
    if file_extension=="jsonl":
        with open(file_path, 'r') as f:
            json_list = []
            for line in f:
                json_list.append(json.loads(line))
            return json_list
    else:
        with open(file_path, 'r') as f:
            return json.load(f)

In [ ]:
flask_evaluation = get_json_list('/content/drive/MyDrive/대학교/자연어처리/tool/flask_evaluation.jsonl')

In [ ]:
flask_evaluation

In [ ]:
flask_math = []
for i in range(len(flask_evaluation)):
    if('Math' in flask_evaluation[i]['domain']):
        flask_math.append(flask_evaluation[i])

In [ ]:
flask_math

In [ ]:
json[0]['domain'][0] == 'Humanities'

True

In [ ]:
def run_eval(model_path, model_id, question_file, answer_file, num_gpus):
    # split question file into num_gpus files
    ques_jsons = []
    with open(os.path.expanduser(question_file), "r") as ques_file:
        for line in ques_file:
            ques_jsons.append(line)

    chunk_size = len(ques_jsons) // num_gpus
    ans_handles = []
    for i in range(0, len(ques_jsons), chunk_size):
        ans_handles.append(get_model_answers.remote(model_path, model_id, ques_jsons[i:i + chunk_size]))

    ans_jsons = []
    for ans_handle in ans_handles:
        ans_jsons.extend(ray.get(ans_handle))

    with open(os.path.expanduser(answer_file), "w") as ans_file:
        for line in ans_jsons:
            ans_file.write(json.dumps(line) + "\n")